In [1]:
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


In [2]:
def decode(datum):
    y = np.zeros((datum.shape[0],1))
    for i in range(datum.shape[0]):
        y[i] = np.argmax(datum[i])
    return y
def encode(datum):
    return to_categorical(datum)

In [3]:
np.random.seed(1)
K                    = 10
inner_activation_fun = 'relu'
outer_activation_fun = 'sigmoid'
optimizer_loss_fun   = 'mse'
optimizer_algorithm  = 'adam'
number_inner_layers  = 3
number_inner_neurons = 256
number_epoch         = 200
batch_length         = 10
show_inter_results   = 2

In [4]:
print("Loading Data ...")
Data = np.loadtxt(r"RF_Data.csv", delimiter=",")

Loading Data ...


In [5]:
print("Preparing Data ...")
x = np.transpose(Data[0:2047,:])
Label_1 = np.transpose(Data[2048:2049,:]); Label_1 = Label_1.astype(int);
Label_2 = np.transpose(Data[2049:2050,:]); Label_2 = Label_2.astype(int);
Label_3 = np.transpose(Data[2050:2051,:]); Label_3 = Label_3.astype(int);
y = encode(Label_3)

Preparing Data ...


In [ ]:
cvscores    = []
cnt         = 0
kfold = StratifiedKFold(n_splits=K, shuffle=True, random_state=1)
for train, test in kfold.split(x, decode(y)):
    cnt = cnt + 1
    print(cnt)
    model = Sequential()
    for i in range(number_inner_layers):
        model.add(Dense(int(number_inner_neurons/2), input_dim = x.shape[1], activation = inner_activation_fun))
    model.add(Dense(y.shape[1], activation = outer_activation_fun))

    print(model.summary())
    model.compile(loss = optimizer_loss_fun, optimizer = optimizer_algorithm, metrics =         ['accuracy'])
    history = model.fit(x[train], y[train], epochs = number_epoch, batch_size = batch_length, verbose = show_inter_results)
    print(history)
    scores = model.evaluate(x[test], y[test], verbose = show_inter_results)
    print(scores[1]*100)
    cvscores.append(scores[1]*100)
    y_pred = model.predict(x[test])
    np.savetxt("Results_3%s.csv" % cnt, np.column_stack((y[test], y_pred)), delimiter=",", fmt='%s')

1
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               262144    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1290      
Total params: 296,458
Trainable params: 296,458
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
 - 10s - loss: 0.0793 - accuracy: 0.2693
Epoch 2/200
 - 10s - loss: 0.0702 - accuracy: 0.3219
Epoch 3/200
 - 9s - loss: 0.0689 - accuracy: 0.3315
Epoch 4/200
 - 10s - loss: 0.0679 - accuracy: 0.3396
Epoch 5/200
 - 10s - loss: 0

Epoch 136/200
 - 10s - loss: 0.0544 - accuracy: 0.4721
Epoch 137/200
 - 10s - loss: 0.0534 - accuracy: 0.4863
Epoch 138/200
 - 10s - loss: 0.0498 - accuracy: 0.5250
Epoch 139/200
 - 10s - loss: 0.0497 - accuracy: 0.5273
Epoch 140/200
 - 10s - loss: 0.0498 - accuracy: 0.5256
Epoch 141/200
 - 10s - loss: 0.0496 - accuracy: 0.5265
Epoch 142/200
 - 10s - loss: 0.0551 - accuracy: 0.4740
Epoch 143/200
 - 10s - loss: 0.0497 - accuracy: 0.5251
Epoch 144/200
 - 10s - loss: 0.0511 - accuracy: 0.5155
Epoch 145/200
 - 10s - loss: 0.0540 - accuracy: 0.4827
Epoch 146/200
 - 10s - loss: 0.0499 - accuracy: 0.5251
Epoch 147/200
 - 10s - loss: 0.0495 - accuracy: 0.5263
Epoch 148/200
 - 9s - loss: 0.0510 - accuracy: 0.5110
Epoch 149/200
 - 10s - loss: 0.0528 - accuracy: 0.4960
Epoch 150/200
 - 10s - loss: 0.0535 - accuracy: 0.4883
Epoch 151/200
 - 10s - loss: 0.0548 - accuracy: 0.4733
Epoch 152/200
 - 10s - loss: 0.0497 - accuracy: 0.5226
Epoch 153/200
 - 10s - loss: 0.0494 - accuracy: 0.5291
Epoch 154/2

 - 10s - loss: 0.0551 - accuracy: 0.4758
Epoch 70/200
 - 10s - loss: 0.0537 - accuracy: 0.4909
Epoch 71/200
 - 11s - loss: 0.0548 - accuracy: 0.4764
Epoch 72/200
 - 10s - loss: 0.0519 - accuracy: 0.5073
Epoch 73/200
 - 10s - loss: 0.0546 - accuracy: 0.4788
Epoch 74/200
 - 10s - loss: 0.0547 - accuracy: 0.4763
Epoch 75/200
 - 10s - loss: 0.0529 - accuracy: 0.4924
Epoch 76/200
 - 9s - loss: 0.0541 - accuracy: 0.4793
Epoch 77/200
 - 10s - loss: 0.0518 - accuracy: 0.5060
Epoch 78/200
 - 11s - loss: 0.0543 - accuracy: 0.4781
Epoch 79/200
 - 10s - loss: 0.0518 - accuracy: 0.5037
Epoch 80/200
 - 10s - loss: 0.0534 - accuracy: 0.4917
Epoch 81/200
 - 10s - loss: 0.0523 - accuracy: 0.5009
Epoch 82/200
 - 10s - loss: 0.0546 - accuracy: 0.4791
Epoch 83/200
 - 10s - loss: 0.0516 - accuracy: 0.5093
Epoch 84/200
 - 10s - loss: 0.0537 - accuracy: 0.4854
Epoch 85/200
 - 10s - loss: 0.0518 - accuracy: 0.5022
Epoch 86/200
 - 10s - loss: 0.0556 - accuracy: 0.4674
Epoch 87/200
 - 10s - loss: 0.0518 - accur

Epoch 2/200
 - 17s - loss: 0.0702 - accuracy: 0.3209
Epoch 3/200
 - 10s - loss: 0.0686 - accuracy: 0.3394
Epoch 4/200
 - 11s - loss: 0.0678 - accuracy: 0.3437
Epoch 5/200
 - 10s - loss: 0.0672 - accuracy: 0.3463
Epoch 6/200
 - 10s - loss: 0.0667 - accuracy: 0.3564
Epoch 7/200
 - 10s - loss: 0.0662 - accuracy: 0.3547
Epoch 8/200
 - 10s - loss: 0.0659 - accuracy: 0.3592
Epoch 9/200
 - 10s - loss: 0.0655 - accuracy: 0.3630
Epoch 10/200
 - 10s - loss: 0.0653 - accuracy: 0.3661
Epoch 11/200
 - 10s - loss: 0.0649 - accuracy: 0.3713
Epoch 12/200
 - 10s - loss: 0.0647 - accuracy: 0.3721
Epoch 13/200
 - 10s - loss: 0.0645 - accuracy: 0.3728
Epoch 14/200
 - 10s - loss: 0.0642 - accuracy: 0.3780
Epoch 15/200
 - 10s - loss: 0.0642 - accuracy: 0.3784
Epoch 16/200
 - 10s - loss: 0.0639 - accuracy: 0.3786
Epoch 17/200
 - 10s - loss: 0.0637 - accuracy: 0.3836
Epoch 18/200
 - 10s - loss: 0.0636 - accuracy: 0.3848
Epoch 19/200
 - 10s - loss: 0.0634 - accuracy: 0.3868
Epoch 20/200
 - 9s - loss: 0.0629 - 

 - 10s - loss: 0.0504 - accuracy: 0.5161
Epoch 154/200
 - 10s - loss: 0.0495 - accuracy: 0.5296
Epoch 155/200
 - 10s - loss: 0.0494 - accuracy: 0.5243
Epoch 156/200
 - 10s - loss: 0.0582 - accuracy: 0.4393
Epoch 157/200
 - 11s - loss: 0.0514 - accuracy: 0.5097
Epoch 158/200
 - 10s - loss: 0.0556 - accuracy: 0.4633
Epoch 159/200
 - 10s - loss: 0.0616 - accuracy: 0.4306
Epoch 160/200
 - 10s - loss: 0.0527 - accuracy: 0.5068
Epoch 161/200
 - 10s - loss: 0.0521 - accuracy: 0.5031
Epoch 162/200
 - 10s - loss: 0.0510 - accuracy: 0.5137
Epoch 163/200
 - 10s - loss: 0.0512 - accuracy: 0.5073
Epoch 164/200
 - 10s - loss: 0.0555 - accuracy: 0.4691
Epoch 165/200
 - 10s - loss: 0.0549 - accuracy: 0.4738
Epoch 166/200
 - 11s - loss: 0.0548 - accuracy: 0.4767
Epoch 167/200
 - 10s - loss: 0.0520 - accuracy: 0.5036
Epoch 168/200
 - 10s - loss: 0.0514 - accuracy: 0.5110
Epoch 169/200
 - 11s - loss: 0.0509 - accuracy: 0.5141
Epoch 170/200
 - 11s - loss: 0.0502 - accuracy: 0.5166
Epoch 171/200
 - 11s - l

Epoch 87/200
 - 10s - loss: 0.0514 - accuracy: 0.5087
Epoch 88/200
 - 10s - loss: 0.0519 - accuracy: 0.5062
Epoch 89/200
 - 10s - loss: 0.0513 - accuracy: 0.5100
Epoch 90/200
 - 10s - loss: 0.0518 - accuracy: 0.5096
Epoch 91/200
 - 10s - loss: 0.0511 - accuracy: 0.5110
Epoch 92/200
 - 10s - loss: 0.0505 - accuracy: 0.5167
Epoch 93/200
 - 10s - loss: 0.0559 - accuracy: 0.4634
Epoch 94/200
 - 10s - loss: 0.0507 - accuracy: 0.5149
Epoch 95/200
 - 10s - loss: 0.0519 - accuracy: 0.5041
Epoch 96/200
 - 10s - loss: 0.0506 - accuracy: 0.5159
Epoch 97/200
 - 10s - loss: 0.0504 - accuracy: 0.5169
Epoch 98/200
 - 10s - loss: 0.0503 - accuracy: 0.5174
Epoch 99/200
 - 10s - loss: 0.0500 - accuracy: 0.5157
Epoch 100/200
 - 10s - loss: 0.0507 - accuracy: 0.5137
Epoch 101/200
 - 10s - loss: 0.0518 - accuracy: 0.5007
Epoch 102/200
 - 9s - loss: 0.0501 - accuracy: 0.5197
Epoch 103/200
 - 9s - loss: 0.0501 - accuracy: 0.5163
Epoch 104/200
 - 10s - loss: 0.0499 - accuracy: 0.5218
Epoch 105/200
 - 10s - lo

Epoch 20/200
 - 10s - loss: 0.0634 - accuracy: 0.3873
Epoch 21/200
 - 10s - loss: 0.0629 - accuracy: 0.3914
Epoch 22/200
 - 10s - loss: 0.0628 - accuracy: 0.3952
Epoch 23/200
 - 9s - loss: 0.0616 - accuracy: 0.4093
Epoch 24/200
 - 9s - loss: 0.0591 - accuracy: 0.4383
Epoch 25/200
 - 10s - loss: 0.0578 - accuracy: 0.4468
Epoch 26/200
 - 10s - loss: 0.0570 - accuracy: 0.4563
Epoch 27/200
 - 10s - loss: 0.0559 - accuracy: 0.4739
Epoch 28/200
 - 10s - loss: 0.0559 - accuracy: 0.4690
Epoch 29/200
 - 10s - loss: 0.0576 - accuracy: 0.4523
Epoch 30/200
 - 10s - loss: 0.0568 - accuracy: 0.4595
Epoch 31/200
 - 10s - loss: 0.0571 - accuracy: 0.4559
Epoch 32/200
 - 10s - loss: 0.0582 - accuracy: 0.4406
Epoch 33/200
 - 11s - loss: 0.0566 - accuracy: 0.4618
Epoch 34/200
 - 10s - loss: 0.0567 - accuracy: 0.4634
Epoch 35/200
 - 10s - loss: 0.0556 - accuracy: 0.4725
Epoch 36/200
 - 10s - loss: 0.0585 - accuracy: 0.4449
Epoch 37/200
 - 10s - loss: 0.0564 - accuracy: 0.4661
Epoch 38/200
 - 10s - loss: 0.

Epoch 171/200
 - 9s - loss: 0.0494 - accuracy: 0.5268
Epoch 172/200
 - 11s - loss: 0.0491 - accuracy: 0.5277
Epoch 173/200
 - 10s - loss: 0.0492 - accuracy: 0.5265
Epoch 174/200
 - 11s - loss: 0.0559 - accuracy: 0.4574
Epoch 175/200
 - 10s - loss: 0.0544 - accuracy: 0.4745
Epoch 176/200
 - 10s - loss: 0.0507 - accuracy: 0.5159
Epoch 177/200
